In [13]:
import cv2 as cv
import numpy as np

# ========== Cargar imagen ==========
image = cv.imread('images/cats.png')
if image is None:
    raise FileNotFoundError("No se encontró 'images/cats.png'")
h, w = image.shape[:2]

# Lienzo amplio para evitar recortes
canvas_w, canvas_h = w * 2, h * 2
offset_x, offset_y = (canvas_w - w) // 2, (canvas_h - h) // 2

# Centro de rotación (en coordenadas del canvas de entrada)
cx, cy = float(offset_x + w // 2), float(offset_y + h // 2)

# Matriz afín del último frame (salida <- entrada)
M_display = np.array([[1.0, 0.0, 0.0],
                      [0.0, 1.0, 0.0]], dtype=np.float32)

# ========== UI ==========
cv.namedWindow("Transformaciones", cv.WINDOW_NORMAL)

def nothing(_): pass

# Rango cómodo de traslación: [-1000, +1000]
TR_RANGE = 2000
TR_ZERO  = TR_RANGE // 2

cv.createTrackbar("Tx", "Transformaciones", TR_ZERO, TR_RANGE, nothing)
cv.createTrackbar("Ty", "Transformaciones", TR_ZERO, TR_RANGE, nothing)
cv.createTrackbar("Angulo", "Transformaciones", 0, 360, nothing)
cv.createTrackbar("EscalaX", "Transformaciones", 100, 300, nothing)  # 1.00 = 100
cv.createTrackbar("EscalaY", "Transformaciones", 100, 300, nothing)
cv.createTrackbar("Uniforme", "Transformaciones", 0, 1, nothing)

print("Usa los sliders para transformar la imagen.")
print("Haz CLIC IZQUIERDO donde quieras fijar el centro de giro (la imagen no se moverá).")
print("ESC para salir.")

# ========== Callback del ratón ==========
def mouse_callback(event, x, y, flags, param):
    """Fija un nuevo centro sin mover la imagen: compensa ajustando Tx/Ty."""
    global cx, cy, M_display
    if event == cv.EVENT_LBUTTONDOWN:
        # salida -> entrada
        M_inv = cv.invertAffineTransform(M_display)
        src_x = M_inv[0,0]*x + M_inv[0,1]*y + M_inv[0,2]
        src_y = M_inv[1,0]*x + M_inv[1,1]*y + M_inv[1,2]

        # nuevo centro en el espacio de entrada
        cx, cy = float(src_x), float(src_y)

        # ajustar traslación para que el punto clicado siga coincidiendo visualmente
        tx_prime = int(round(x - cx))
        ty_prime = int(round(y - cy))
        cv.setTrackbarPos("Tx", "Transformaciones", np.clip(tx_prime + TR_ZERO, 0, TR_RANGE))
        cv.setTrackbarPos("Ty", "Transformaciones", np.clip(ty_prime + TR_ZERO, 0, TR_RANGE))

cv.setMouseCallback("Transformaciones", mouse_callback)

# ========== Bucle principal ==========
while True:
    # Canvas de entrada con la imagen centrada
    canvas = np.zeros((canvas_h, canvas_w, 3), dtype=np.uint8)
    canvas[offset_y:offset_y+h, offset_x:offset_x+w] = image

    # Leer controles
    tx = cv.getTrackbarPos("Tx", "Transformaciones") - TR_ZERO
    ty = cv.getTrackbarPos("Ty", "Transformaciones") - TR_ZERO
    angle_deg = cv.getTrackbarPos("Angulo", "Transformaciones")
    sx = max(cv.getTrackbarPos("EscalaX", "Transformaciones") / 100.0, 1e-4)
    sy = max(cv.getTrackbarPos("EscalaY", "Transformaciones") / 100.0, 1e-4)
    if cv.getTrackbarPos("Uniforme", "Transformaciones") == 1:
        sy = sx

    # Matrices homogéneas
    rad = np.deg2rad(angle_deg)
    c, s = np.cos(rad), np.sin(rad)

    T1 = np.array([[1, 0, -cx],
                   [0, 1, -cy],
                   [0, 0,  1]], dtype=np.float32)  # Llevar pivote a origen

    S  = np.array([[sx, 0, 0],
                   [0, sy, 0],
                   [0,  0, 1]], dtype=np.float32)  # Escalado

    R  = np.array([[ c, -s, 0],
                   [ s,  c, 0],
                   [ 0,  0, 1]], dtype=np.float32)  # Rotación

    T2 = np.array([[1, 0, cx],
                   [0, 1, cy],
                   [0, 0,  1]], dtype=np.float32)  # Devolver pivote

    T3 = np.array([[1, 0, tx],
                   [0, 1, ty],
                   [0, 0,  1]], dtype=np.float32)  # Traslación global

    M = T3 @ T2 @ R @ S @ T1
    M_affine = M[:2, :]
    M_display = M_affine.copy()  # Para el clic

    # Aplicar transformación
    out = cv.warpAffine(canvas, M_affine, (canvas_w, canvas_h),
                        flags=cv.INTER_LINEAR,
                        borderMode=cv.BORDER_CONSTANT, borderValue=(0,0,0))

    # Dibujar solo el punto de rotación (rojo sólido)
    px = int(round(M_affine[0,0]*cx + M_affine[0,1]*cy + M_affine[0,2]))
    py = int(round(M_affine[1,0]*cx + M_affine[1,1]*cy + M_affine[1,2]))
    cv.circle(out, (px, py), 6, (0, 0, 255), -1)

    # Mostrar
    cv.imshow("Transformaciones", out)
    if cv.waitKey(10) & 0xFF == 27:
        break

cv.destroyAllWindows()


Usa los sliders para transformar la imagen.
Haz CLIC IZQUIERDO donde quieras fijar el centro de giro (la imagen no se moverá).
ESC para salir.
